In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as funcs
from pyspark.sql import SQLContext
from pyspark import Row
import os
from pyspark.sql.functions import udf
from  pyspark.sql.types import BooleanType

In [2]:
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark-client"
conf = SparkConf()
conf.setMaster('yarn-client')
conf.setAppName('viktor')
conf.set("spark.executor.memory", "8g")
conf.set("spark.executor.cores", "2")
conf.set("spark.executor.instances", "5")

In [3]:
sc.stop()

NameError: name 'sc' is not defined

In [3]:
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [16]:
a_df = sqlContext.read.parquet("/tdg/2017/07/18/paths")

In [17]:
a_df.show(1)

+----+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----+----------------------+
|IMSI|          Trajectory|          crm_fields|        other_fields|              labels|             metrics|extrapolation|speed|trajectory_extra_field|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----+----------------------+
|   0|[WrappedArray(120...|Map(gender -> 2, ...|Map(overnight_zip...|Map(mode -> Map()...|Map(duration -> 0...|         null| null|  Map(trip_last_cel...|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----+----------------------+
only showing top 1 row



In [18]:
def passed_from_poi(pois):
    def meth(trajectory):
        node_ids = set([node_tuple[0] for node_tuple in trajectory])
        for node in pois:
            if node in node_ids:
                return True
        return False
    return meth

In [25]:
check_trajectory = udf(passed_from_poi(set([59908733])), BooleanType())
count = a_df.filter(check_trajectory(a_df.Trajectory)).select('IMSI').distinct().count()
print count

34515


In [23]:
check_trajectory = udf(passed_from_poi(set([3744344742])), BooleanType())
count = a_df.filter(check_trajectory(a_df.Trajectory)).select('IMSI').distinct().count()


In [24]:
print count

33310


In [37]:

ddf = sqlContext.read.parquet("/tdg/2017/07/18/aggregated_metrics/directions/tiles/tile_direction_hours")

In [11]:
df.show(1, False)

+-------+-------------------------+--------+---------+-------+--------+--------+---------+-------+--------+--------------+---------------+-------------+--------------+--------------+---------------+-------------+--------------+
|tile_id|ts_start                 |north_in|north_out|east_in|east_out|south_in|south_out|west_in|west_out|north_in_speed|north_out_speed|east_in_speed|east_out_speed|south_in_speed|south_out_speed|west_in_speed|west_out_speed|
+-------+-------------------------+--------+---------+-------+--------+--------+---------+-------+--------+--------------+---------------+-------------+--------------+--------------+---------------+-------------+--------------+
|157829 |2017-07-18T20:00:00+02:00|0       |0        |2      |0       |0       |2        |0      |0       |0.0           |0.0            |0.0          |0.0           |0.0           |0.0            |0.0          |0.0           |
+-------+-------------------------+--------+---------+-------+--------+--------+--------

In [15]:
ddf.where("tile_id = 5888785").select(["south_out", "north_out", "east_out", "west_out"]).show(30)

+---------+---------+--------+--------+
|south_out|north_out|east_out|west_out|
+---------+---------+--------+--------+
|        0|        0|       0|    2342|
|        0|        0|       0|    2777|
|        0|        0|       0|    4515|
|        0|        0|       0|    2377|
|        0|        0|       0|   10020|
|        0|        0|       0|    7625|
|        0|        0|       0|    2337|
|        0|        0|       0|    9870|
|        0|        0|       0|    7495|
|        0|        0|       0|    9142|
|        0|        0|       0|    9585|
|        0|        0|       0|   10462|
|        0|        0|       0|    2047|
|        0|        0|       0|    3650|
|        0|        0|       0|    5882|
|        0|        0|       0|    1705|
|        0|        0|       0|    9695|
|        0|        0|       0|    8542|
|        0|        0|       0|    4572|
|        0|        0|       0|   10835|
|        0|        0|       0|    8952|
|        0|        0|       0|    9225|


In [38]:
ddf.where("tile_id = 5888785").select(["south_in", "north_in", "east_in", "west_in"]).show(30)

+--------+--------+-------+-------+
|south_in|north_in|east_in|west_in|
+--------+--------+-------+-------+
|    2342|       0|      0|      0|
|    2772|       0|      0|      0|
|    4510|       0|      0|      0|
|    2380|       0|      0|      0|
|   10017|       0|      0|      0|
|    7635|       0|      0|      0|
|    2337|       0|      0|      0|
|    9870|       0|      0|      0|
|    7487|       0|      0|      0|
|    9137|       0|      0|      0|
|    9575|       0|      0|      0|
|   10450|       0|      0|      0|
|    2047|       0|      0|      0|
|    3650|       0|      0|      0|
|    5877|       0|      0|      0|
|    1697|       0|      0|      0|
|    9697|       0|      0|      0|
|    8540|       0|      0|      0|
|    4585|       0|      0|      0|
|   10842|       0|      0|      0|
|    8957|       0|      0|      0|
|    9237|       0|      0|      0|
|   10297|       0|      0|      0|
|    9785|       0|      0|      0|
+--------+--------+-------+-

In [26]:

df = sqlContext.read.parquet("/tdg/2017/07/18/aggregated_metrics/segments/")

In [27]:
df.show(1, False)

+----+--------------------------+-----------+-------------+----------------------------------------------------------------------------------------------------------------------+----------+-----+------------------+----------+
|IMSI|crm_fields                |destination|extrapolation|other_fields                                                                                                          |source    |speed|tile_ids          |ts_start  |
+----+--------------------------+-----------+-------------+----------------------------------------------------------------------------------------------------------------------+----------+-----+------------------+----------+
|0   |Map(gender -> 2, age -> 4)|414066185  |2.5          |Map(overnight_zip -> null, source -> Nexus CS, record_type -> IuCS MOC, clean_IMSI -> -1, tac -> 35740007, mcc -> 262)|1206618164|-1.0 |[4465449, 4461977]|1500390705|
+----+--------------------------+-----------+-------------+-------------------------------------

In [28]:
print df.where("source = 59908733").select('IMSI').distinct().count()
#59908733  3744344742

34500


In [29]:
print df.where("source = 3744344742").select('IMSI').distinct().count()

33302


In [39]:
df.where("source = 3744344742").show(1, False)

+----+----------------+-----------+-------------+---------------------------------------------------------------------------------------------------------------------------------+----------+------------------+---------------------------+----------+
|IMSI|crm_fields      |destination|extrapolation|other_fields                                                                                                                     |source    |speed             |tile_ids                   |ts_start  |
+----+----------------+-----------+-------------+---------------------------------------------------------------------------------------------------------------------------------+----------+------------------+---------------------------+----------+
|129 |Map(dummy -> -1)|154830     |2.5          |Map(overnight_zip -> null, source -> Nexus CS, record_type -> AIF/Update Location, clean_IMSI -> -1, tac -> 01399900, mcc -> 208)|3744344742|113.09555032910242|[5888784, 5888785, 5885313]|1500388673|
+---

In [33]:
 df.where("source = 59908733").show(1, False)

+----+----------------+-----------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------+--------+-----------------+---------------------------+----------+
|IMSI|crm_fields      |destination|extrapolation|other_fields                                                                                                                               |source  |speed            |tile_ids                   |ts_start  |
+----+----------------+-----------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------+--------+-----------------+---------------------------+----------+
|442 |Map(dummy -> -1)|2753774100 |2.5          |Map(overnight_zip -> null, source -> Nexus PS, rac -> 1, record_type -> Gb PDP Context Activation, clean_IMSI -> -1, tac -> -1, mcc -> 230)|59908733|59.76977758745933|[5892256, 588878

In [34]:

tdf = sqlContext.read.parquet("/tdg/2017/07/18/aggregated_metrics/tiles/")

In [35]:
tdf.show(1, False)

+----+--------------------------+-------------+----------------------------------------------------------------------------------------------------------------------+-------+----------+
|IMSI|crm_fields                |extrapolation|other_fields                                                                                                          |tile_id|ts_start  |
+----+--------------------------+-------------+----------------------------------------------------------------------------------------------------------------------+-------+----------+
|0   |Map(gender -> 2, age -> 4)|2.5          |Map(overnight_zip -> null, source -> Nexus CS, record_type -> IuCS MOC, clean_IMSI -> -1, tac -> 35740007, mcc -> 262)|4465449|1500390705|
+----+--------------------------+-------------+----------------------------------------------------------------------------------------------------------------------+-------+----------+
only showing top 1 row



In [36]:
tdf.where("tile_id = 5888785").select('IMSI').distinct().count()

57954

In [21]:
df = df.withColumn("in_total", df.north_in + df.east_in + df.south_in + df.west_in)
df = df.withColumn("out_total", df.north_out + df.east_out + df.south_out + df.west_out)


In [22]:
df = df.cache()

In [23]:
tiles = df.where("in_total > 100").select("tile_id").collect()

In [24]:
result = []
for t in tiles:
    result.append(t.tile_id)

In [25]:
print result[::100]

[189090, 4433177, 3392762, 3053644, 1677901, 4820685, 2362245, 2654077, 2022209, 921958, 3817265, 813614, 6188157, 4168347, 2252215, 2445495, 2098185, 5951277, 6333534, 3060053, 3327918, 1911134, 1862644, 6323117, 6691185, 3078250, 2379821, 1407212, 4977048, 4321186, 3838385, 4175503, 2361899, 1987770, 799794, 1591162, 4264591, 2650226, 2091983, 2008432, 5663399, 3504537, 2172291, 1914740, 4803335, 2625656, 2265963, 1466098, 3911382, 1886828, 4152943, 2866508, 1890454, 4318549, 3063257, 1244444, 1025887, 3178433, 1845109, 5268663, 3539728, 1330977, 4803336, 2522160, 5070680, 3059890, 1227092, 5140428, 2372566, 1161701, 3015741, 1460117, 5569715, 2810039, 1032982, 3660580, 2522160, 1244606, 3063256, 3191891, 3060093, 1765513, 6462012, 4810269, 4803334, 2827472, 1875421, 1362053, 7234593, 4216330, 3437892, 1727364, 2008432, 1091445, 4294232, 2504608, 1618979, 4325527, 4100973, 2341672, 1362054, 4244524, 3503854, 1424486, 1150432, 3191405, 2431895, 5753449, 4810270, 1970260, 1522592, 3258

In [26]:
tr_tiles = [3964588, 3787192, 3826701, 3734814, 3731342, 2615522, 3050299, 3046827, 1970530, 1967057, 2587555, 2580612, 4064944, 4965470, 4962518, 4959046, 7116476, 1472349, 2170691, 7048399, 3400642, 2629385, 1029630, 1663411, 1289537, 3968025, 2577070, 2573598, 4012758, 4325499, 4322026, 2118587, 3300231, 3289814, 2573923, 4465501, 2282459, 2282458, 6524151, 2252207, 2248735, 2354973, 2354974, 3316394, 2355543, 4861313, 1355872, 1765790, 924816, 928288, 1546526, 1504870, 5778809, 880064, 5684144, 769131, 769132, 3907878, 3907879, 955681, 955680, 2643386, 2639915, 5403224, 5385865, 1706172, 3695740, 3695739, 3649667, 3262799, 3262798, 7040061, 2467170, 2470642, 4271677, 4054589, 4051117, 3606229, 3616644, 3901309, 3897839, 3999010, 3995538, 2646593, 2643121, 2630883, 2634354, 3529228, 3525756, 4068409, 2143447, 1928385, 4998418, 4078289, 4074817, 6722055, 6718582, 5482584, 5482585, 1019119, 5763850, 3783721, 3974837, 3971365, 5822828, 5808946, 1313354, 901211, 897738, 3732085, 1803177, 1799704, 4142476, 4139005, 5656465, 6010289, 3434722, 3434723, 5646367, 5642896, 1675303, 3560579, 2418008, 6326146, 6326147, 3612465, 3430217, 1244606, 5528125, 5524654, 2893057, 2893055, 1362816, 2129734, 1758289, 3266230, 3266231, 4026325, 4246776, 1612863, 1703159, 1710102, 2577284, 2573812, 3032384, 3028912, 1750360, 1746888, 2389998, 1907551, 1563745, 1560273, 5520885, 1689482, 1686011, 1983923, 4718002, 4718003, 2623436, 914870, 5243651, 5240179, 3754278, 3754281, 2626909, 1078067, 3596204, 3596203, 3663822, 2778332, 2605135, 324502, 2942278, 1334992, 1334993, 1934488, 1931016, 3910806, 3910804, 1984461, 1984460, 4049542, 4042598, 3511760, 1671831, 6291511, 1769017, 1765545, 3370060, 1757189, 1737458, 1307062, 1307060, 2018845, 2008431, 2230934, 3421090, 3421089, 4754265, 4754264, 2019449, 2012504, 4467906, 1650670, 3768521, 3765049, 5847234, 5843763, 2182434, 904335, 904336, 3929365, 1379333, 2091156, 2942026, 2938553, 2737312, 2737313, 2557908, 1553351, 2313415, 2313414, 3986962, 3990431, 3281831, 3278359, 1328110, 1331582, 4363088, 4370032, 1549880, 373361, 723913, 727387, 1411284, 1411283, 3300606, 2498565, 2495093, 647580, 644108, 2607893, 2604421, 2903193, 2906665, 1025759, 1022287, 3660349, 4386014, 4386013, 456486, 453014, 5026189, 2799881, 4319425, 4319426, 7475825, 7475826, 1520894, 1321378, 4769839, 2370478, 2370480, 6781571, 6781570, 4240298, 917704, 921175, 376953, 4226187, 4222715, 4208648, 4208649, 6156617, 3038656, 637061, 1227876, 4902817, 4902816, 1282516, 2160481, 2157009, 1279043, 2763978, 5726292, 1237961, 5542420, 5007533, 5004060, 5538947, 2755436, 4026324, 2727636, 5729764, 3290189, 6198671, 6198673, 4164999, 4165001, 1387682, 1988214, 1988213, 6897021, 5408095, 5404622, 6644114, 6654142, 6647198, 7074873, 5053962, 5831099, 5799651, 4602021, 6126546, 6703416, 6703415, 7449661, 7338605, 7394184, 5931890, 5928418, 5779365, 5775894, 6181591, 6129496, 6282456, 6289398, 5627359, 5627358, 5796379, 8230424, 8226952, 7501232, 7504703, 6325124, 6321652, 6856154, 6849201, 4814970, 6217107, 6217106, 6733803, 8182316, 8175371, 7174872, 7803521, 7806998, 7075415, 7453134, 4372650, 4372651, 5407761, 5400816, 4855395, 4858867, 4432112, 4428639, 6903965, 7491627, 7488155, 5195752, 5195751, 5831098, 6130018, 6883761, 6887232, 7335132, 7505363, 7505364, 6120597, 6124069, 6512541, 6509069, 3971521, 3950691, 3734805, 3734803, 1994839, 4071907, 4009408, 4068414, 4005938, 4989765, 5011135, 4969482, 5018068, 7119951, 7106065, 7126889, 7119950, 7116480, 7102591, 2646747, 994914, 1005326, 1289546, 1299966, 2563176, 2552760, 2559701, 2584010, 4016222, 4009283, 2097742, 3237744, 3241216, 2532265, 2553096, 2584328, 4482860, 4475915, 4479389, 4479388, 2296345, 2299816, 2299823, 6489422, 2207058, 2323736, 2334148, 3274730, 3281672, 2359010, 2338177, 4920328, 1352404, 1348936, 1369768, 1595126, 5792682, 935622, 921720, 890485, 5715379, 5743158, 5722323, 5711907, 751779, 727463, 793424, 737891, 803848, 3939145, 3921783, 959156, 2615625, 2626007, 5427509, 3716548, 3307945, 2505374, 2422027, 4365421, 4243905, 4019862, 3644420, 3897848, 3908263, 3918679, 4016381, 4023308, 4012900, 4012897, 4002495, 4019852, 3971241, 2724623, 2672536, 2672538, 2728095, 3518798, 3515327, 3515326, 3529251, 3525779, 1886716, 1900612, 1911025, 1890186, 1914496, 1904080, 4977589, 4026224, 4005391, 6770661, 998293, 963573, 1039937, 994802, 3846218, 3842747, 3967905, 3978320, 3967907, 915102, 1776218, 1755383, 3571003, 3567525, 3570997, 1161276, 1261965, 1178639, 1182110, 1279319, 1189064, 5510739, 5517682, 5517701, 5510737, 5500323, 5535058, 2906944, 2903473, 2903472, 2882644, 2122784, 1817298, 1796459, 1810351, 3311343, 3252370, 4243307, 1560772, 1671904, 1668428, 1675377, 1675375, 2573824, 2976847, 3035853, 1743401, 1739930, 1746872, 1732988, 2403871, 2383074, 1560261, 5534771, 5531299, 5545187, 5541715, 2001277, 4704099, 869755, 928741, 852390, 5243657, 1043356, 1098904, 3571894, 3568425, 3568424, 3571897, 2591245, 2667635, 2615547, 2608603, 2664161, 2636380, 2639852, 355752, 380053, 355753, 2872828, 2865882, 1362778, 1338476, 1328057, 1960151, 3976645, 4028695, 6218582, 3304081, 2001496, 2077858, 2255240, 2199693, 3382914, 3396800, 4660526, 4663995, 4663996, 1953467, 2092378, 4488741, 4488740, 5815997, 5795160, 5812524, 5822941, 2151176, 876556, 904315, 3953684, 3953685, 2177964, 2994101, 1563781, 2302994, 3285309, 3281837, 3271420, 1324633, 1303797, 1303798, 1300327, 4272816, 4446409, 4425583, 4422111, 397665, 404606, 675301, 737805, 1390448, 1480739, 1338377, 2467316, 2463842, 2439543, 609380, 654538, 619813, 623285, 2892762, 2875407, 2910147, 1039639, 4465875, 4451985, 4991470, 4284722, 4333295, 4284721, 7541783, 7538314, 7489701, 1513949, 1517417, 1527850, 4804553, 4222949, 952409, 945466, 352657, 4205343, 4212287, 6142748, 6139275, 6125377, 3101148, 3104620, 4923648, 4923650, 5038782, 5073508, 5076980, 2738071, 2772809, 4002026, 3998549, 4002014, 4002021, 3984660, 3314498, 6212545, 4182344, 4161537, 1932664, 6862287, 6935219, 6872706, 5442811, 5446283, 5435868, 5446282, 6668028, 5102568, 6143920, 6140448, 6126559, 6647885, 6644411, 6894176, 7394180, 7366416, 5723828, 5723829, 7557431, 5754709, 5761654, 5855409, 7362933, 7469980, 7588041, 7518580, 6255698, 6332087, 6384128, 6737294, 6723388, 7216532, 7213059, 7213061, 7793095, 7855587, 7411494, 5439009, 4452947]
tr_tiles = [2018845]

In [27]:
adf = df.select(["tile_id", "in_total", "out_total"]).groupby("tile_id").sum().cache()


In [28]:
adf.toPandas().to_csv("all_agg_hessen_dir_20170411.csv")

In [29]:
import pymongo


In [30]:
from pyspark.sql.functions import col
df.where(col("tile_id").isin(set(tr_tiles))).select(["tile_id", "ts_start", "north_out","east_out", "south_out", "west_out", "north_in","east_in", "south_in", "west_in"]).orderBy(["tile_id", "ts_start"]).show()

+-------+--------------------+---------+--------+---------+--------+--------+-------+--------+-------+
|tile_id|            ts_start|north_out|east_out|south_out|west_out|north_in|east_in|south_in|west_in|
+-------+--------------------+---------+--------+---------+--------+--------+-------+--------+-------+
|2018845|2017-04-11T04:00:...|        0|       0|        0|       0|       0|      0|       0|   1580|
|2018845|2017-04-11T05:00:...|        0|       0|        0|       0|       0|      0|       0|   3322|
|2018845|2017-04-11T06:00:...|        0|       0|        0|       0|       0|      0|       0|   5002|
|2018845|2017-04-11T07:00:...|        0|       0|        0|       0|       0|      0|       0|   7430|
|2018845|2017-04-11T08:00:...|        0|       0|        0|       0|       0|      0|       0|   8420|
|2018845|2017-04-11T09:00:...|        0|       0|        0|       0|       0|      0|       0|   8252|
|2018845|2017-04-11T10:00:...|        0|       0|        0|       0|     

In [31]:
df = df.withColumn("in_total", df.north_in + df.east_in + df.south_in + df.west_in)
df = df.withColumn("out_total", df.north_out + df.east_out + df.south_out + df.west_out)

In [32]:
df = df.withColumn("diff", df.in_total - df.out_total)

In [33]:
df.select("tile_id", "diff", "in_total", "out_total").where(col("tile_id").isin(set(tr_tiles))).orderBy("diff").show(10, False)

+-------+----+--------+---------+
|tile_id|diff|in_total|out_total|
+-------+----+--------+---------+
|2018845|1500|1500    |0        |
|2018845|1580|1580    |0        |
|2018845|2382|2382    |0        |
|2018845|2882|2882    |0        |
|2018845|3322|3322    |0        |
|2018845|4562|4562    |0        |
|2018845|4805|4805    |0        |
|2018845|5002|5002    |0        |
|2018845|5077|5077    |0        |
|2018845|7005|7005    |0        |
+-------+----+--------+---------+
only showing top 10 rows



In [34]:
print 27269434
print 23388690
print 13634717 * 2 - 16540014

27269434
23388690
10729420


In [35]:
df = sqlContext.read.parquet("/tdg/2017/04/11/paths_saarland_minor_P9_heur_200m/").select(["IMSI", "Trajectory"]).cache()

In [36]:
print df.count()

21439381


In [37]:
def passed_from_poi(pois):
    def meth(trajectory):
        node_ids = set([node_tuple[0] for node_tuple in trajectory])
        for node in pois:
            if node in node_ids:
                return True
        return False
    return meth

In [38]:
check_trajectory = udf(passed_from_poi(set([20870876, 20870827])), BooleanType())
print df.filter(check_trajectory(df.Trajectory)).select('IMSI').distinct().count()

48503


In [39]:
check_trajectory = udf(passed_from_poi(set([25263475, 615768651])), BooleanType())
print df.filter(check_trajectory(df.Trajectory)).select('IMSI').distinct().count()

53702


In [ ]:
check_trajectory = udf(passed_from_poi(set([25266308, 254347285])), BooleanType())
print df.filter(check_trajectory(df.Trajectory)).select('IMSI').distinct().count()

38111


In [ ]:
check_trajectory = udf(passed_from_poi(set([25263926, 25263927])), BooleanType())
print df.filter(check_trajectory(df.Trajectory)).select('IMSI').distinct().count()

In [ ]:
check_trajectory = udf(passed_from_poi(set([25266310, 2367662793])), BooleanType())
print df.filter(check_trajectory(df.Trajectory)).select('IMSI').distinct().count()

In [11]:

check_trajectory = udf(passed_from_poi(set([819061663, 20870913])), BooleanType())
print df.filter(check_trajectory(df.Trajectory)).select('IMSI').distinct().count()

30954


In [12]:
check_trajectory = udf(passed_from_poi(set([25263019, 3753196379])), BooleanType())
print df.filter(check_trajectory(df.Trajectory)).select('IMSI').distinct().count()

46338


In [13]:
check_trajectory = udf(passed_from_poi(set([389653, 266100770])), BooleanType())
print df.filter(check_trajectory(df.Trajectory)).select('IMSI').distinct().count()

67743


In [ ]:
segments = [(835557717,3546228402),(323109076,718512),(246948888,571170931),(4749290686,283651224),(279405778,279405868),(2149418620,759819083),(2149418624,12450403),(4008414232,503247),(36318641,389654),(389653,266100770),(819061663,20870913),(355268772,480573),(469388,466968),(1585021802,21464514),(12453150,1585021784),(2567579072,2414252289),(2567579075,2567579073),(279264025,279263618),(25263926,25263927),(3282289168,1649719690),(398265,398267),(1563697917,1563697936),(1563697928,1563697908),(20865747,291921472),(14790947,291921473),(2803998282,2803998284),(2803998281,3873520199),(10705270,10705239),(499848,1672628699),(464068,464069),(1559388572,4745043863),(4749288206,305400779),(21420604,2895860639),(12454118,2895860638),(20966554,2086290110),(2086290170,2388675),(60013396,4743682863),(4743682864,316824187),(3656779067,528731940)]

for seg in segments:
    check_trajectory = udf(passed_from_poi(set(seg)), BooleanType())
    count = df.filter(check_trajectory(df.Trajectory)).select('IMSI').distinct().count()
    print seg[0], count